In [39]:
#importing packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from datetime import datetime
import csv
from bs4 import BeautifulSoup

In [40]:
#define url
def get_url(position,location):
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position,location)
    return url

In [41]:
#tags to search for
url = get_url('baker', 'florida')

In [42]:
response = requests.get(url)

In [43]:
#fiinding post that fir criteria
soup = BeautifulSoup(response.text, 'html.parser')
cards =soup.find_all('div', 'jobsearch-SerpJobCard')

In [44]:
#selecting info from each post
card = cards[0]
atag = card.h2.a
job_title = atag.get('title')
job_url = 'https://www.indeed.com/' + atag.get('href')
company = card.find('span', 'company').text.strip()
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
job_summary = card.find('div','summary').text.strip()
post_date = card.find('span', 'date').text
today = datetime.today().strftime('%Y-%m-%d')
try:
    job_salary = card.find('span', 'salaryText').text.strip()
except AttributeError:
    job_salary = ''

In [45]:
#extracting info into a card
def get_record(card):
    atag = card.h2.a
    job_title = atag.get('title')
    job_url = 'https://www.indeed.com/' + atag.get('href')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    job_summary = card.find('div','summary').text.strip()
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span', 'salaryText').text.strip()
    except AttributeError:
        job_salary = ''
    record = (job_title, job_location, post_date, today, job_summary, job_salary, job_url)
    return record

records =[]
for card in cards:
    record =get_record(card)
    records.append(record)

In [57]:
#searching in the different pages
while True:
    try:
        url = 'https://www.indeed.com/' + soup.find('a',{'aria-label':'Next'}).get('href')
    except AttributeError:
        break
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards =soup.find_all('div', 'jobsearch-SerpJobCard')
    for card in cards:
        record =get_record(card)
        records.append(record)

In [63]:
#extracting everything 
import requests
from datetime import datetime
import csv
from bs4 import BeautifulSoup


def get_url(position,location):
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position,location)
    return url

def get_record(card):
    atag = card.h2.a
    job_title = atag.get('title')
    job_url = 'https://www.indeed.com/' + atag.get('href')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    job_summary = card.find('div','summary').text.strip()
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span', 'salaryText').text.strip()
    except AttributeError:
        job_salary = ''
    record = (job_title, job_location, post_date, today, job_summary, job_salary, job_url)
    return record

def main(position,location):
    records=[]
    url=get_url(position, location)
    
    #extract job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards =soup.find_all('div', 'jobsearch-SerpJobCard')
    
        for card in cards:
            record =get_record(card)
            records.append(record) 
        try:
            url = 'https://www.indeed.com/' + soup.find('a',{'aria-label':'Next'}).get('href')
        except AttributeError:
            break
    
    with open('bakers_florida.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job_title', 'Job_location', 'post_date', 'today_date', 'Job_summary', 'Job_salary', 'url'])
        writer.writerows(records)
    
  

In [64]:
main('baker','florida')

In [66]:
bakers = pd.read_csv('bakers_florida.csv')
bakers.head()

,Job_title,Job_location,post_date,today_date,Job_summary,Job_salary,url
0,Baker,"Orlando, FL",Today,2020-12-30,The Baker is responsible for overall product i...,NaN,https://www.indeed.com//rc/clk?jk=f8c022749ebf...
1,Baker,"Bradenton, FL",2 days ago,2020-12-30,Texas Roadhouse is looking for a Baker who bel...,NaN,https://www.indeed.com//rc/clk?jk=f237147d60d7...
2,CAFE BAKER,"Merritt Is, FL",30+ days ago,2020-12-30,Job Description School/Center: Edgewood Junior...,NaN,https://www.indeed.com//rc/clk?jk=56b67813c572...
3,Food Service Cook-Baker,"Vero Beach, FL",28 days ago,2020-12-30,Knowledge of quantity cooking and ability to f...,$186 a day,https://www.indeed.com//rc/clk?jk=182a32180b16...
4,Part-Time Bakery Clerk Job,"Homestead, FL",Today,2020-12-30,"Packages, displays, rotates, and stores bakery...",NaN,https://www.indeed.com//rc/clk?jk=c5dd6a92708b...
